In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313503 sha256=2f485cc655b0b3bf8536e4e50a56ea86b6ecbd5004a2c984f81d626cb8e51580
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/merged1.csv     
  inflating: dataset/merged10.csv    
  inflating: dataset/merged11.csv    
  inflating: dataset/merged12.csv    
  inflating: dataset/merged13.csv    
  inflating: dataset/merged14.csv    
  inflating: dataset/merged15.csv    
  inflating: dataset/merged16.csv    
  inflating: dataset/merged17.csv    
  inflating: dataset/merged18.csv    
  inflating: dataset/merged19.csv    
  inflating: dataset/merged2.csv     
  inflating: dataset/merged20.csv    
  inflating: dataset/merged21.csv    
  inflating: dataset/merged22.csv    
  inflating: dataset/merged3.csv     
  inflating: dataset/merged4.csv     
  inflating: dataset/merged5.csv     
  inflating: dataset/merged6.csv     
  inflating: dataset/merged7.csv     
  inflating: dataset/merged8.csv     
  inflating: dataset/merged9.csv     


In [ ]:
import pandas as pd
import os

dataset_folder = rf"dataset"

all_data = []

for file in os.listdir(dataset_folder):
    if file.endswith(".csv"):
        dataset_path = os.path.join(dataset_folder, file)

        df = pd.read_csv(dataset_path)

        if "merged_col" in df.columns and "cScore" in df.columns:
            df_sampled = df[["merged_col", "cScore"]].sample(n=min(8000, len(df)))
            all_data.append(df_sampled)
            print(f"✅ Loaded: {file} ({len(df_sampled)} rows)")
        else:
            print(f"⚠️ Skipping {file}: Missing required columns.")

df = pd.concat(all_data, ignore_index=True)

✅ Loaded: merged19.csv (8000 rows)
✅ Loaded: merged17.csv (1171 rows)
✅ Loaded: merged12.csv (918 rows)
✅ Loaded: merged1.csv (235 rows)
✅ Loaded: merged11.csv (8000 rows)
✅ Loaded: merged2.csv (4705 rows)
✅ Loaded: merged6.csv (1643 rows)
✅ Loaded: merged7.csv (2658 rows)
✅ Loaded: merged22.csv (5075 rows)
✅ Loaded: merged14.csv (8000 rows)
✅ Loaded: merged18.csv (8000 rows)
✅ Loaded: merged16.csv (545 rows)
✅ Loaded: merged5.csv (8000 rows)
✅ Loaded: merged8.csv (8000 rows)
✅ Loaded: merged10.csv (8000 rows)
✅ Loaded: merged3.csv (65 rows)
✅ Loaded: merged20.csv (963 rows)
✅ Loaded: merged21.csv (8000 rows)
✅ Loaded: merged15.csv (8000 rows)
✅ Loaded: merged13.csv (8000 rows)
✅ Loaded: merged9.csv (198 rows)
✅ Loaded: merged4.csv (8000 rows)


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove non-alphabetic characters and lowercase the text
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower()
    return text

# Apply text cleaning to the dataset
df['cleaned_text'] = df['merged_col'].apply(clean_text)

# Display cleaned text
df[['merged_col', 'cleaned_text']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,merged_col,cleaned_text
0,2.5 BHK flat for sale at SAPPHIRE HEIGHTS w...,bhk flat for sale at sapphire heights with...
1,Builder floor apartment is available for sale....,builder floor apartment is available for sale ...
2,"Multistorey Apartment for Sale in Zirakpur, Ch...",multistorey apartment for sale in zirakpur cha...
3,"nan 2 BHK, Multistorey Apartment is available ...",nan bhk multistorey apartment is available fo...
4,"Multistorey Apartment for Sale in Sargasan, Ga...",multistorey apartment for sale in sargasan gan...


In [ ]:
import numpy as np

def generate_nlp_features(df):
    # Character count
    df['char_count'] = df['cleaned_text'].apply(len)
    # Word count
    df['word_count'] = df['cleaned_text'].apply(lambda x: len(x.split()))
    # Average word length
    df['avg_word_length'] = df['cleaned_text'].apply(lambda x: np.mean([len(word) for word in x.split()]))
    # Stopword count
    df['stopword_count'] = df['cleaned_text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
    # Hashtag count (for datasets like tweets)
    df['htag_count'] = df['cleaned_text'].apply(lambda x: x.count('#'))

    return df

# Generate NLP features
df = generate_nlp_features(df)
df[['char_count', 'word_count', 'avg_word_length', 'stopword_count', 'htag_count']].head()

,char_count,word_count,avg_word_length,stopword_count,htag_count
0,203,31,5.387097,8,0
1,180,30,4.933333,10,0
2,174,25,5.840000,8,0
3,81,13,5.076923,4,0
4,175,25,5.880000,8,0


In [ ]:
!pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import fasttext
import fasttext.util

# Bag of Words
bow_vectorizer = CountVectorizer(max_features=5000)
X_bow = bow_vectorizer.fit_transform(df['cleaned_text'])

# TFIDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_text'])

# FastText embeddings (example using pre-trained FastText model)
#fasttext.util.download_model('en', if_exists='ignore')
ft_model = fasttext.load_model('cc.en.300.bin')

def get_fasttext_embeddings(text):
    words = text.split()
    word_vectors = [ft_model.get_word_vector(word) for word in words]
    return np.mean(word_vectors, axis=0)

df['fasttext_embeddings'] = df['cleaned_text'].apply(get_fasttext_embeddings)

--2025-03-16 13:26:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.164.78.72, 18.164.78.128, 18.164.78.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.164.78.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   170MB/s    in 26s     

2025-03-16 13:26:41 (168 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split data
X = df[['char_count', 'word_count', 'avg_word_length', 'stopword_count', 'htag_count']]
y = df['cScore']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model (Random Forest as an example)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.6331700885289132


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'BoW Accuracy: {accuracy_score(y_test, y_pred)}')

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'TFIDF Accuracy: {accuracy_score(y_test, y_pred)}')

X_train, X_test, y_train, y_test = train_test_split(np.vstack(df['fasttext_embeddings']), y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'FastText Accuracy: {accuracy_score(y_test, y_pred)}')

BoW Accuracy: 0.7293746468261443
TFIDF Accuracy: 0.7295630062158599
FastText Accuracy: 0.726360896590695


In [ ]:
from scipy.sparse import hstack

# Combine NLP features and TFIDF features
X_combined = hstack([X, X_tfidf])

# Train model and evaluate
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Combined Features Accuracy: {accuracy_score(y_test, y_pred)}')

Combined Features Accuracy: 0.7363439442456207


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, 'text_classification_model.pkl')

# Load the model for predictions
loaded_model = joblib.load('text_classification_model.pkl')

# Example: Make predictions on real-world input (e.g., a new text)
new_text = ["I bought a house in pune for rs. 1000000 which is 4 bhk."]
new_text_cleaned = [clean_text(text) for text in new_text]

# Generate NLP features for the new text
new_text_df = pd.DataFrame({'cleaned_text': new_text_cleaned})  # Create a DataFrame
new_text_df = generate_nlp_features(new_text_df)  # Apply the feature generation function

# TFIDF features for the new text
new_text_tfidf = tfidf_vectorizer.transform(new_text_cleaned)

# Combine NLP and TFIDF features for the new text
X_new = new_text_df[['char_count', 'word_count', 'avg_word_length', 'stopword_count', 'htag_count']]
X_new_combined = hstack([X_new, new_text_tfidf])  # Combine features

# Predict using the trained model
prediction = loaded_model.predict(X_new_combined)
print(f'Prediction for new text: {prediction}')

Prediction for new text: ['medium']
